In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as func
from scipy.io import loadmat
import numpy as np
from sklearn.feature_extraction import image
import pandas as pd
from nltk.corpus import stopwords, wordnet as wn
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize
import string
import gensim

In [2]:
import re
import pickle
import json
import xml.etree.ElementTree as ET
from nltk.corpus import stopwords, wordnet as wn
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import os
from nltk.tokenize import word_tokenize, sent_tokenize
import numpy as np
import math
from difflib import SequenceMatcher
import pandas as pd

In [3]:
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()
year_regex = re.compile(r'((19[0-9]{2})|(20[0-9]{2}))[a-z]?')
conversion_dict = {}
stop_words = [',', '.', '(', ')', ':', '-', "+", ";", "a", "about", "al", "al.", "all", 
	"already", "also", "although", "am", "an", "and", "another", "any", "anyhow", "are", 
	"aren", "aren't", "around", "as", "at", "back", "be", "because", "been", 
	"being", "beyond", "but", "by", "can", "cannot", "cant", "co", "con", "could", "couldn", 
	"couldnt", "d", "de", "did", "didn", "didn't", "do", "does", "doesn", "doesn't", 
	"doing", "don", "don't", "done", "due", "each", "either", "else", "elsewhere", "et", 
	"etc", "even", "ever", "except", "for", "found", "from", "further", "had", "hadn", 
	"hadn't", "has", "hasn", "hasn't", "hasnt", "have", "haven", "haven't", "having", 
	"he", "hence", "her", "here", "hereafter", "hereby", "hers", 
	"herself", "him", "himself", "his", "how", "however", "i", "ie", "if", "in", "inc", 
	"indeed", "interest", "into", "is", "isn", "isn't", "it", "it's", "its", "itself", 
	"just", "ltd", "ll", "m", "may", "me", "meanwhile", "might", "mightn", 
	"mightn't", "mine", "moreover", "most", "mostly", "move", "much", "must", "mustn", 
	"mustn't", "my", "myself", "name", "namely", "need", "needn", "needn't", "neither", 
	"nevertheless", "no", "nobody", "noone", "nor", "not", "now", "nowhere", "o", "of", 
	"off", "often", "on", "only", "onto", "or", "other", "others", "otherwise", "our", "own", 
	"per", "perhaps", "put", "rather", "re", "s", "same", "see", "seem", "seemed", 
	"seeming", "seems", "serious", "she", "should", "shouldn", "shouldn't", "since", 
	"sincere", "so", "some", "somehow", "someone", "something", "somewhere", "still", 
	"such", "t", "take", "than", "that", "that'll", "the", "their", "theirs", 
	"them", "themselves", "then", "thence", "there", "thereafter", "thereby", "therefore", 
	"therein", "thereupon", "these", "they", "this", "those", "though", "throughout", 
	"thru", "thus", "to", "together", "too", "toward", "towards", "un", "until", "upon", 
	"us", "ve", "very", "via", "was", "wasn", "wasn't", "we", "well", "were", "weren", 
	"weren't", "what", "whatever", "when", "whence", "whenever", "where", "whereafter", 
	"whereas", "whereby", "wherein", "whereupon", "wherever", "whether", "which", "while", 
	"whither", "who", "whoever", "whole", "whom", "whose", "why", "will", "with", "within", 
	"without", "won", "won't", "would", "wouldn", "wouldn't", "y", "yet", "you", "your", 
	"yours", "yourself", "yourselves", "from SVM import SVCone", "two", "three", "four", "five", "six", "seven",
	"eight", "nine", "zero", "between", 'below', 'ourselves', "you'll", 'again', 'once', 'over', 'shan', 'few', 
    'against', 'before', 'out', 'down', 'both', 'up', "you've", "shan't", "you're", "should've", 'ours', 'ma', 
    "couldn't", 'during', 'more', 'ain', 'through', 'after', 'above', "she's", "you'd", 'under' ]

In [4]:
def get_citations(folder):
    citation_list = {}
    for file in os.listdir(folder) :
        tree = ET.parse(folder+file)
        root = tree.getroot()
        id = file[:8]
        for element in root.iterfind("algorithm"):
            if(element.attrib['name']=="ParsCit"):
                citlist = element.getchildren()
                cits = citlist[0].getchildren()
                citations = []
                for cit in cits:
                    cit_dict = {}
                    if(cit.attrib['valid']=="true"):
                        try :
                            title = cit.find('title').text.lower()
                        except :
                            title = cit.find('rawString').text.lower()
                        cit_dict['title'] = title
                        cit_dict['cit'] = cit
                        citations.append(cit_dict)
                
                citation_list[id] = citations
            
    return citation_list 

In [5]:
citations = get_citations("../xmls/")

In [6]:
def get_words(string, cit_auths=None):

    string = string.replace('-', '').lower()
    context_words = word_tokenize(string)
    final_context_words = []

    for word in context_words:
        if word in stop_words:
            continue
        if re.fullmatch(year_regex, word): #or match_auths(word, cit_auths):
            continue
        if re.fullmatch(r'[0-9]+([.][0-9]+)?', word):
            word = '<number>'
            final_context_words.append(word)
        elif 'this_citation' in word:
            final_context_words.append('<this_citation>')
        elif re.fullmatch(r'[a-z]+', word):
            original_word = word
            final_word = lemmatizer.lemmatize(word)
            
            if final_word not in conversion_dict:
                conversion_dict[final_word] = {}

            if original_word not in conversion_dict[final_word]:
                conversion_dict[final_word][original_word] = 0
            conversion_dict[final_word][original_word]+=1

            final_context_words.append(final_word)

    return final_context_words

In [7]:
def get_contexts(citations) :
    dataset = {}
    count = 0
    for key in citations.keys():
        context_list = []
        for cit in citations[key] :
            dict1 = {}
            dict1['paper_name'] = cit['title']
            dict1['context'] = ""
            for context in cit['cit'].findall('contexts/context') :
                text = context.text.lower()
                citstr = context.get('citStr').lower()
                text = text.replace(citstr, "this_citation")
                dict1['context']+= text+" "
            dict1['context'] = get_words(dict1['context'])
            context_list.append(dict1)
        dataset[key] = context_list
        count+=1
        if(count%100==0) :
            print(count)
    return dataset

In [8]:
contexts = get_contexts(citations)

100
200
300
400
500
600
700
800
900
1000
1100


In [9]:
max1 = 0
avg = 0
number = 0
for key in contexts :
    papers = contexts[key]
    for pap in papers :
        val = len(pap['context'])
        avg += val
        number += 1
        if(val>max1) :
            max1 = val
            
print(max1)
print(avg/number)

1141
140.41749366058292


In [10]:
embeddings_dict = {}
with open("glove.6B.100d.txt", 'r', encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], "float32")
        embeddings_dict[word] = vector

In [11]:
print(len(embeddings_dict.keys()))

400000


In [12]:
mapping = {}
mapping_rev = {}
values = []
count = 0
for key in contexts :
    papers = contexts[key]
    for pap in papers :
        name = pap['paper_name']
        words = pap['context']
        if(len(words)>0) :
            mapping[key+"_|_"+name] = count
            mapping_rev[count] = key+"_|_"+name
            count+=1
            embed = []
            for word in words :
                if(word in embeddings_dict) :
                    embed.append(embeddings_dict[word])
                else :
                    embed.append(embeddings_dict['unk'])
            diff = 1141-len(words)
            if(diff>0) :
                for i in range(diff) :
                    embed.append(embeddings_dict['unk'])
            values.append(embed)  

In [13]:
print(len(values))

27635


In [14]:
tags = pickle.load(open("../pickles_data/baseline_tags.pkl","rb"))

In [15]:
output = []
for i in range(count) :
    key,name = mapping_rev[i].split("_|_")
    papers = tags[key]
    for pap in papers :
        if(pap['paper_name']==name) :
            output.append(pap['tag'])
            break

In [16]:
print(len(output))

27635


In [17]:
size = len(values)
trainset = values[:int(0.8*size)]
trainlabel = output[:int(0.8*size)]
testset = values[int(0.8*size):]
testlabel = output[int(0.8*size):]

In [18]:
def shuffle(train_data, train_output) :
    baselines = []
    non_baselines = []
    for i in range(len(train_output)) :
        if(train_output[i]==1) :
            baselines.append(train_data[i])
        else :
            non_baselines.append(train_data[i])
    
    n = len(baselines)
    ar = np.random.choice(len(non_baselines), len(baselines))
    nb_ar = []
    for x in ar :
        nb_ar.append(non_baselines[x])
        
    data = []
    data.extend(nb_ar)
    output = []
    for i in range(len(data)):
        output.append(0)
    
    data.extend(baselines)
    for i in range(n) :
        output.append(1)
        
    total_data = []
    for i in range(len(data)):
        ar = []
        ar.append(data[i])
        ar.append(output[i])
        total_data.append(ar)
        
    total_data = np.array(total_data)
    np.random.shuffle(total_data)
    
    data = []
    output = []
    for ar in total_data :
        data.append(ar[0])
        output.append(ar[1])
        
    data = np.array(data)
    output = np.array(output)
    
    return data, output

In [19]:
# trainset, trainlabel = shuffle(trainset, trainlabel)

In [20]:
from torch.autograd import Variable
from torch.nn import functional as F

class RCNN(nn.Module):
	def __init__(self, batch_size, output_size, hidden_size, embedding_length):
		super(RCNN, self).__init__()
		
		"""
		Arguments
		---------
		batch_size : Size of the batch which is same as the batch_size of the data returned by the TorchText BucketIterator
		output_size : 2 = (pos, neg)
		hidden_sie : Size of the hidden_state of the LSTM
		vocab_size : Size of the vocabulary containing unique words
		embedding_length : Embedding dimension of GloVe word embeddings
		weights : Pre-trained GloVe word_embeddings which we will use to create our word_embedding look-up table 
		
		"""
		
		self.batch_size = batch_size
		self.output_size = output_size
		self.hidden_size = hidden_size
# 		self.vocab_size = vocab_size
		self.embedding_length = embedding_length
		
# 		self.word_embeddings = nn.Embedding(vocab_size, embedding_length)# Initializing the look-up table.
# 		self.word_embeddings.weight = nn.Parameter(weights, requires_grad=False) # Assigning the look-up table to the pre-trained GloVe word embedding.
		self.dropout = 0.8
		self.lstm = nn.LSTM(embedding_length, hidden_size, dropout=self.dropout, bidirectional=True)
		self.W2 = nn.Linear(2*hidden_size+embedding_length, hidden_size)
		self.label = nn.Linear(hidden_size, output_size)
		
	def forward(self, input1, batch_size=None):
	
		""" 
		Parameters
		----------
		input_sentence: input_sentence of shape = (batch_size, num_sequences)
		batch_size : default = None. Used only for prediction on a single sentence after training (batch_size = 1)
		
		Returns
		-------
		Output of the linear layer containing logits for positive & negative class which receives its input as the final_hidden_state of the LSTM
		final_output.shape = (batch_size, output_size)
		
		"""
		
		"""
		
		The idea of the paper "Recurrent Convolutional Neural Networks for Text Classification" is that we pass the embedding vector
		of the text sequences through a bidirectional LSTM and then for each sequence, our final embedding vector is the concatenation of 
		its own GloVe embedding and the left and right contextual embedding which in bidirectional LSTM is same as the corresponding hidden
		state. This final embedding is passed through a linear layer which maps this long concatenated encoding vector back to the hidden_size
		vector. After this step, we use a max pooling layer across all sequences of texts. This converts any varying length text into a fixed
		dimension tensor of size (batch_size, hidden_size) and finally we map this to the output layer.
		"""
# 		input = self.word_embeddings(input_sentence) # embedded input of shape = (batch_size, num_sequences, embedding_length)
		input1 = input1.permute(1, 0, 2) # input.size() = (num_sequences, batch_size, embedding_length)
		if batch_size is None:
			h_0 = Variable(torch.zeros(2, self.batch_size, self.hidden_size).cuda()) # Initial hidden state of the LSTM
			c_0 = Variable(torch.zeros(2, self.batch_size, self.hidden_size).cuda()) # Initial cell state of the LSTM
		else:
			h_0 = Variable(torch.zeros(2, batch_size, self.hidden_size).cuda())
			c_0 = Variable(torch.zeros(2, batch_size, self.hidden_size).cuda())

		output, (final_hidden_state, final_cell_state) = self.lstm(input1, (h_0, c_0))
		
		final_encoding = torch.cat((output, input1), 2).permute(1, 0, 2)
		y = self.W2(final_encoding) # y.size() = (batch_size, num_sequences, hidden_size)
		y = y.permute(0, 2, 1) # y.size() = (batch_size, hidden_size, num_sequences)
		y = F.max_pool1d(y, y.size()[2]) # y.size() = (batch_size, hidden_size, 1)
		y = y.squeeze(2)
		logits = self.label(y)
		
		return logits

In [21]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
batch_size = 64

train_data = TensorDataset(torch.as_tensor(trainset), torch.as_tensor(trainlabel))
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

val_data = TensorDataset(torch.as_tensor(testset), torch.as_tensor(testlabel))
val_sampler = RandomSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=64)

In [22]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report

def eval_model(model, val_loader) :
    final_out = []
    final_lab = []

    for idx, (val_input, val_label) in enumerate(val_loader):
    # val_input = val_input.permute(1,0,2)
    # print(train_input.shape)
        if(val_input.shape[0]==64) :
            val_input = val_input.type(torch.FloatTensor)
            val_input = val_input.cuda()
            output = model(val_input)
            output = output.cpu().detach().numpy()
            val_label = val_label.cpu().detach().numpy()
            ar= []
            for i in range(output.shape[0]) :
                index = -1
                max_val = -1
                for j in range(len(output[i])) :
                    if(output[i][j]>max_val) :
                        index = j
                        max_val = output[i][j]
                ar.append(index)

            val_label = list(val_label)
            final_out.extend(ar)
            final_lab.extend(val_label)

        
    return classification_report(final_lab, final_out)

In [ ]:
learning_rate = 0.005
num_epochs = 10
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()
model = RCNN(64,2,32,100).to(device)
weights = [1.0, 5.0]
class_weights = torch.FloatTensor(weights).cuda()
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    for batch_id, (train_input, train_label) in enumerate(train_dataloader):
        if(train_input.shape[0]==64) :
            optimizer.zero_grad() 
            train_input = train_input.type(torch.FloatTensor)
            train_input = train_input.cuda()
            output = model(train_input)

            train_label = train_label.type(torch.LongTensor)
            train_label = train_label.cuda()
            loss = criterion(output, train_label)
            loss.backward()
            optimizer.step()

            if batch_id % 100 == 0:
                print('Epoch [{}/{}], Loss:{:.4f}'
                    .format(epoch+1, num_epochs, loss.data))
                print("classification_report")
                print(eval_model(model,val_dataloader))
                print("--------------------------------------------------------------")
                # losses.append(loss.data)

            del train_input
            del train_label
            del output
            del loss
            torch.cuda.empty_cache()

/home/tanisha17116/.local/lib/python3.6/site-packages/torch/nn/modules/rnn.py:50: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.8 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


Epoch [1/10], Loss:0.7513
classification_report
              precision    recall  f1-score   support

           0       0.91      0.68      0.78      4979
           1       0.11      0.37      0.17       525

    accuracy                           0.65      5504
   macro avg       0.51      0.52      0.47      5504
weighted avg       0.83      0.65      0.72      5504

--------------------------------------------------------------
Epoch [1/10], Loss:0.6528
classification_report
              precision    recall  f1-score   support

           0       0.96      0.83      0.89      4979
           1       0.30      0.69      0.42       525

    accuracy                           0.82      5504
   macro avg       0.63      0.76      0.66      5504
weighted avg       0.90      0.82      0.85      5504

--------------------------------------------------------------
Epoch [1/10], Loss:0.3994
classification_report
              precision    recall  f1-score   support

           0       0.

Epoch [5/10], Loss:0.1724
classification_report
              precision    recall  f1-score   support

           0       0.97      0.92      0.94      4978
           1       0.47      0.69      0.56       526

    accuracy                           0.89      5504
   macro avg       0.72      0.80      0.75      5504
weighted avg       0.92      0.89      0.90      5504

--------------------------------------------------------------
Epoch [6/10], Loss:0.1507
classification_report
              precision    recall  f1-score   support

           0       0.96      0.92      0.94      4979
           1       0.46      0.66      0.55       525

    accuracy                           0.89      5504
   macro avg       0.71      0.79      0.74      5504
weighted avg       0.92      0.89      0.90      5504

--------------------------------------------------------------
Epoch [6/10], Loss:0.1454
classification_report
              precision    recall  f1-score   support

           0       0.

In [40]:
print(eval_model(model, val_dataloader))

              precision    recall  f1-score   support

           0       0.94      0.97      0.96      4978
           1       0.62      0.40      0.49       526

    accuracy                           0.92      5504
   macro avg       0.78      0.69      0.72      5504
weighted avg       0.91      0.92      0.91      5504



In [ ]:
torch.save(model.state_dict, "models/2layer_bilstm_atten_statedict.pt")

In [ ]:
torch.save(model, "models/2layer_bilstm_atten_model.pt")

In [ ]:
class Attention_Net_Features(nn.Module) :
    
    